Notebook 01 Goals:
Combine and clean the raw datasets into one consistent dataset ready for EDA (exploratory analysis).
This includes:
    - Loading all raw datasets
    - Normalizing identifiers (like CustomerID)
    - Merging the tables correctly
    - Handling missing values and data types
    - Saving the cleaned dataset to data/interim/

1. Importing the libraries/functions:

In [1]:
import pandas as pd
import sys, numpy as np
import os, json, re
from pathlib import Path
from sklearn.impute import SimpleImputer


2. Paths and names of the files

XX. Saving the clean data frame

In [2]:
save_df(df_clean, name="telco_master_clean", stage="processed", fmt="parquet")

NameError: name 'save_df' is not defined